In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pygeos
from momepy import COINS, CircularCompactness
from shapely.geometry import LineString, Point
from shapely.ops import linemerge, polygonize

/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# importing functions

In [2]:
def _polygonize_ifnone(edges, polys):
    if polys is None:
        pre_polys = polygonize(edges.geometry)
        polys = gpd.GeoDataFrame(geometry=[g for g in pre_polys], crs=edges.crs)
    return polys


def _selecting_rabs_from_poly(
    gdf, circom_threshold=0.7, area_threshold=0.85, include_adjacent=True
):
    """
    From a GeoDataFrame of polygons, returns a GDF of polygons that are
    above the Circular Compaactness threshold.

    Return
    ________
    GeoDataFrames : round abouts and adjacent polygons
    """
    # calculate parameters
    gdf.loc[:, "area"] = gdf.geometry.area
    circom_serie = CircularCompactness(gdf, "area").series

    # selecting round about polygons based on compactness
    rab = gdf[circom_serie > circom_threshold]
    # exclude those above the area threshold
    area_threshold_val = gdf.area.quantile(area_threshold)
    rab = rab[rab["area"] < area_threshold_val]

    if include_adjacent is True:
        # calculating some parameters
        bounds = rab.geometry.bounds
        rab = pd.concat([rab, bounds], axis=1)
        rab["deltax"] = rab.maxx - rab.minx
        rab["deltay"] = rab.maxy - rab.miny
        rab["rab_diameter"] = rab[["deltax", "deltay"]].max(axis=1)

        # selecting the adjacent areas that are of smaller than itself
        if GPD_10:
            rab_adj = gpd.sjoin(gdf, rab, predicate="intersects")
        else:
            rab_adj = gpd.sjoin(gdf, rab, op="intersects")
        rab_adj = rab_adj[rab_adj.area_right >= rab_adj.area_left]
        rab_adj.index.name = "index"
        rab_adj["hdist"] = 0

        # adding a hausdorff_distance threshold
        # TODO: (should be a way to verctorize)
        for i, group in rab_adj.groupby("index_right"):
            for g in group.itertuples():
                hdist = g.geometry.hausdorff_distance(rab.loc[i].geometry)
                rab_adj.loc[g.Index, "hdist"] = hdist

        rab_plus = rab_adj[rab_adj.hdist < rab_adj.rab_diameter]

    else:
        rab["index_right"] = rab.index
        rab_plus = rab

    # only keeping relevant fields
    geom_col = rab_plus.geometry.name
    rab_plus = rab_plus[[geom_col, "index_right"]]

    return rab_plus


def _rabs_center_points(gdf, center_type="centroid"):
    """
    From a selection of roundabouts, returns an aggregated GeoDataFrame
    per round about with extra column with center_type.
    """
    # creating a multipolygon per RAB (as opposed to dissolving) of the entire
    # composition of the RAB
    # temporary DataFrame where geometry is the array of pygeos geometries
    # Hack until shapely 2.0 is out.
    tmp = pd.DataFrame(gdf.copy())  # creating a copy avoids warnings
    tmp["geometry"] = tmp.geometry.values.data

    pygeos_geoms = (
        tmp.groupby("index_right")
        .geometry.apply(pygeos.multipolygons)
        .rename("geometry")
    )
    pygeos_geoms = pygeos.make_valid(pygeos_geoms)

    rab_multipolygons = gpd.GeoDataFrame(pygeos_geoms, crs=gdf.crs)
    # make_valid is transforming the multipolygons into geometry collections because of
    # shared edges

    if center_type == "centroid":
        # geometry centroid of the actual circle
        rab_multipolygons["center_pt"] = gdf[
            gdf.index == gdf.index_right
        ].geometry.centroid

    elif center_type == "mean":
        coords, idxs = pygeos.get_coordinates(pygeos_geoms, return_index=True)
        means = {}
        for i in np.unique(idxs):
            tmps = coords[idxs == i]
            target_idx = rab_multipolygons.index[i]
            means[target_idx] = Point(tmps.mean(axis=0))

        rab_multipolygons["center_pt"] = gpd.GeoSeries(means, crs=gdf.crs)

    # centerpoint of minimum_bounding_circle
    # TODO
    # # minimun_bounding_circle() should be available in Shapely 2.0. Implementation still
    # pending.

    return rab_multipolygons


def _coins_filtering_many_incoming(incoming_many, angle_threshold=0):
    """
    Used only for the cases when more than one incoming line touches the
    roundabout.
    """
    idx_out_many_incoming = []
    # For each new connection, evaluate COINS and select the group from which the new
    # line belongs
    # TODO ideally use the groupby object on line_wkt used earlier
    for g, x in incoming_many.groupby("line_wkt"):
        gs = gpd.GeoSeries(pd.concat([x.geometry, x.line]), crs=incoming_many.crs)
        gdf = gpd.GeoDataFrame(geometry=gs)
        gdf.drop_duplicates(inplace=True)

        coins = COINS(gdf, angle_threshold=angle_threshold)
        group_series = coins.stroke_attribute()
        gdf["coins_group"] = group_series
        # selecting the incoming and its extension
        coins_group_filter = gdf.groupby("coins_group").count() == 1
        f = gdf.coins_group.map(coins_group_filter.geometry)
        idxs_remove = gdf[f].index
        idx_out_many_incoming.extend(idxs_remove)

    incoming_many_reduced = incoming_many.drop(idx_out_many_incoming, axis=0)

    return incoming_many_reduced


def _selecting_incoming_lines(rab_multipolygons, edges, angle_threshold=0):
    """Selecting only the lines that are touching but not covered by
    the ``rab_plus``.
    If more than one LineString is incoming to ``rab_plus``, COINS algorithm
    is used to select the line to be extended further.
    """
    # selecting the lines that are touching but not covered by
    if GPD_10:
        touching = gpd.sjoin(edges, rab_multipolygons, predicate="touches")
        edges_idx, rabs_idx = rab_multipolygons.sindex.query_bulk(
            edges.geometry, predicate="covered_by"
        )
    else:
        touching = gpd.sjoin(edges, rab_multipolygons, op="touches")
        edges_idx, rabs_idx = rab_multipolygons.sindex.query_bulk(
            edges.geometry, op="covered_by"
        )
    idx_drop = edges.index.take(edges_idx)
    touching_idx = touching.index
    ls = list(set(touching_idx) - set(idx_drop))

    incoming = touching.loc[ls]

    # figuring out which ends of incoming edges needs to be connected to the center_pt
    incoming["first_pt"] = incoming.geometry.apply(lambda x: Point(x.coords[0]))
    incoming["dist_fisrt_pt"] = incoming.center_pt.distance(incoming.first_pt)
    incoming["last_pt"] = incoming.geometry.apply(lambda x: Point(x.coords[-1]))
    incoming["dist_last_pt"] = incoming.center_pt.distance(incoming.last_pt)
    lines = []
    for i, row in incoming.iterrows():
        if row.dist_fisrt_pt < row.dist_last_pt:
            lines.append(LineString([row.first_pt, row.center_pt]))
        else:
            lines.append(LineString([row.last_pt, row.center_pt]))
    incoming["line"] = gpd.GeoSeries(lines, index=incoming.index, crs=edges.crs)

    # checking in there are more than one incoming lines arriving to the same point
    # which would create several new lines
    incoming["line_wkt"] = incoming.line.to_wkt()
    grouped_lines = incoming.groupby(["line_wkt"])["line_wkt"]
    count_s = grouped_lines.count()

    # separating the incoming roads that come on their own to those that come in groups
    filter_count_one = pd.DataFrame(count_s[count_s == 1])
    filter_count_many = pd.DataFrame(count_s[count_s > 1])
    incoming_ones = pd.merge(
        incoming, filter_count_one, left_on="line_wkt", right_index=True, how="inner"
    )
    incoming_many = pd.merge(
        incoming, filter_count_many, left_on="line_wkt", right_index=True, how="inner"
    )
    incoming_many_reduced = _coins_filtering_many_incoming(
        incoming_many, angle_threshold=angle_threshold
    )

    incoming_all = gpd.GeoDataFrame(
        pd.concat([incoming_ones, incoming_many_reduced]), crs=edges.crs
    )

    return incoming_all, idx_drop


def _ext_lines_to_center(edges, incoming_all, idx_out):
    """
    Extends the Linestrings geometries to the centerpoint defined by
    _rabs_center_points. Also deletes the lines that originally defined the roundabout.

    Returns
    -------
    GeoDataFrame
        GeoDataFrame of with updated geometry
    """

    incoming_all["geometry"] = incoming_all.apply(
        lambda row: linemerge([row.geometry, row.line]), axis=1
    )

    # deleting the original round about edges
    new_edges = edges.drop(idx_out, axis=0)

    # mantianing the same gdf shape that the original
    incoming_all = incoming_all[edges.columns]
    new_edges = pd.concat([new_edges, incoming_all])

    return new_edges


def _ext_lines_to_center(edges, incoming_all, idx_out):
    """
    Extends the Linestrings geometries to the centerpoint defined by
    _rabs_center_points. Also deletes the lines that originally defined the roundabout.

    Returns
    -------
    GeoDataFrame
        GeoDataFrame of with updated geometry
    """

    incoming_all["geometry"] = incoming_all.apply(
        lambda row: linemerge([row.geometry, row.line]), axis=1
    )

    # deleting the original round about edges
    new_edges = edges.drop(idx_out, axis=0)

    # creating a unique label for returned gdf
    ls = ["rab_" + x for x in incoming_all.index_right.apply(str)]
    incoming_label = pd.Series(ls, index=incoming_all.index)
    incoming_label = incoming_label[~incoming_label.index.duplicated(keep="first")]

    # mantianing the same gdf shape that the original
    incoming_all = incoming_all[edges.columns]
    new_edges = pd.concat([new_edges, incoming_all])

    # adding a new column to match
    new_edges["simpl_edge"] = np.nan
    new_edges["simpl_edge"] = incoming_label

    return new_edges

# testing

In [8]:
GPD_10 = True

In [4]:
# loading data from file
mad = gpd.read_file(
    "/Users/gregoriomaya/Desktop/GSoc_2022/gsoc2022_network_simpl/data/madrid2062.gpkg"
)
mad.set_index(["u", "v", "key"], inplace=True)

mad_polys = polygonize(mad.geometry)
mad_polys = gpd.GeoDataFrame(geometry=[g for g in mad_polys], crs=mad.crs)

In [51]:
mad_polys["my_area"] = mad_polys.geometry.area

In [52]:
rab_plus = _selecting_rabs_from_poly(mad_polys, area_col = 'my_area')
rab_plus.tail()

,geometry,index_right
index,,
15810,"POLYGON ((605353.740 656290.203, 605351.886 65...",15809
15811,"POLYGON ((605450.314 656422.762, 605450.811 65...",15811
15820,"POLYGON ((599310.998 656796.065, 599307.324 65...",15820
15848,"POLYGON ((602865.024 656746.199, 602869.344 65...",15848
15852,"POLYGON ((592098.635 643109.424, 592098.226 64...",15852


In [198]:
rab_multipolygons = _rabs_center_points(rab_plus)

In [199]:
incoming_all, idx_drop = _selecting_incoming_lines(rab_multipolygons, mad)

In [200]:
output = _ext_lines_to_center(mad, incoming_all, idx_drop)

/Users/gregoriomaya/mambaforge/envs/test/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [47]:
def _selecting_rabs_from_poly(
    gdf,
    area_col="area",
    circom_threshold=0.7,
    area_threshold=0.85,
    include_adjacent=True,
):
    """
    From a GeoDataFrame of polygons, returns a GDF of polygons that are
    above the Circular Compaactness threshold.

    Return
    ________
    GeoDataFrames : round abouts and adjacent polygons
    """
    # calculate parameters
    if area_col == "area":
        gdf.loc[:, area_col] = gdf.geometry.area    
    circom_serie = CircularCompactness(gdf, area_col).series
    # selecting round about polygons based on compactness
    mask = circom_serie > circom_threshold
    rab = gdf[mask]
    # exclude those above the area threshold
    area_threshold_val = gdf.area.quantile(area_threshold)
    rab = rab[rab[area_col] < area_threshold_val]

    if include_adjacent is True:

        # calculating some parameters
        bounds = rab.geometry.bounds
        rab = pd.concat([rab, bounds], axis=1)
        rab["deltax"] = rab.maxx - rab.minx
        rab["deltay"] = rab.maxy - rab.miny
        rab["rab_diameter"] = rab[["deltax", "deltay"]].max(axis=1)

        # selecting the adjacent areas that are of smaller than itself
        if GPD_10:
            rab_adj = gpd.sjoin(gdf, rab, predicate="intersects")
        else:
            rab_adj = gpd.sjoin(gdf, rab, op="intersects")

        area_right = area_col + "_right"
        area_left = area_col + "_left"
        area_mask = rab_adj[area_right] >= rab_adj[area_left]
        rab_adj = rab_adj[area_mask]
        rab_adj.index.name = "index"

        # adding a hausdorff_distance threshold
        rab_adj["hdist"] = 0
        # TODO: (should be a way to verctorize)
        for i, group in rab_adj.groupby("index_right"):
            for g in group.itertuples():
                hdist = g.geometry.hausdorff_distance(rab.loc[i].geometry)
                rab_adj.loc[g.Index, "hdist"] = hdist

        rab_plus = rab_adj[rab_adj.hdist < rab_adj.rab_diameter]

    else:
        rab["index_right"] = rab.index
        rab_plus = rab

    # only keeping relevant fields
    geom_col = rab_plus.geometry.name
    rab_plus = rab_plus[[geom_col, "index_right"]]

    return rab_plus

In [44]:
gdf = mad_polys.copy()

area_col = "my_area"
circom_threshold = 0.7
area_threshold = 0.85
include_adjacent = True

In [45]:
# calculate parameters
gdf.loc[:, area_col] = gdf.geometry.area
circom_serie = CircularCompactness(gdf, area_col).series
# selecting round about polygons based on compactness
mask = circom_serie > circom_threshold
rab = gdf[mask]
# exclude those above the area threshold
area_threshold_val = gdf.area.quantile(area_threshold)
rab = rab[rab[area_col] < area_threshold_val]

if include_adjacent is True:

    # calculating some parameters
    bounds = rab.geometry.bounds
    rab = pd.concat([rab, bounds], axis=1)
    rab["deltax"] = rab.maxx - rab.minx
    rab["deltay"] = rab.maxy - rab.miny
    rab["rab_diameter"] = rab[["deltax", "deltay"]].max(axis=1)

    # selecting the adjacent areas that are of smaller than itself
    if GPD_10:
        rab_adj = gpd.sjoin(gdf, rab, predicate="intersects")
    else:
        rab_adj = gpd.sjoin(gdf, rab, op="intersects")

    area_right = area_col + "_right"
    area_left = area_col + "_left"
    area_mask = rab_adj[area_right] >= rab_adj[area_left]
    rab_adj = rab_adj[area_mask]
    rab_adj.index.name = "index"

    # adding a hausdorff_distance threshold
    rab_adj["hdist"] = 0
    # TODO: (should be a way to verctorize)
    for i, group in rab_adj.groupby("index_right"):
        for g in group.itertuples():
            hdist = g.geometry.hausdorff_distance(rab.loc[i].geometry)
            rab_adj.loc[g.Index, "hdist"] = hdist

    rab_plus = rab_adj[rab_adj.hdist < rab_adj.rab_diameter]

else:
    rab["index_right"] = rab.index
    rab_plus = rab

# only keeping relevant fields
geom_col = rab_plus.geometry.name
rab_plus = rab_plus[[geom_col, "index_right"]]

In [46]:
rab_plus.tail()

,geometry,index_right
index,,
15810,"POLYGON ((605353.740 656290.203, 605351.886 65...",15809
15811,"POLYGON ((605450.314 656422.762, 605450.811 65...",15811
15820,"POLYGON ((599310.998 656796.065, 599307.324 65...",15820
15848,"POLYGON ((602865.024 656746.199, 602869.344 65...",15848
15852,"POLYGON ((592098.635 643109.424, 592098.226 64...",15852
